## Project Objective and Hypotheses

The **goal** of this project is to explore the key factors that influence:

1. The price of used vehicles
2. The time it takes to sell a vehicle (represented by the days_listed column)

**Hypotheses:**

1. Vehicles with higher mileage (odometer) tend to have lower prices
2. Newer vehicles (model_year) are generally more expensive
3. Cars in better condition are both priced higher and sell more quickly
4. Vehicle type influences pricing — e.g., SUVs and trucks may be priced higher than sedans.
5. Fuel type (e.g., electric, hybrid, diesel) affects both price and how quickly vehicles sell.

This analysis aims to support data-driven pricing strategies and inventory management decisions for a used car marketplace.

The dataset contains information about vehicle listings collected from a U.S. online car marketplace.

## 1. Data Overview

We begin by loading the dataset and taking a preliminary look at its structure.  
This helps us understand the available data and spot any early issues.

### 1.1 Loading the Dataset

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Load dataset
df = pd.read_csv('../vehicles_us.csv')

# Preview the first 10 rows
df.head(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28
5,14990,2014.0,chrysler 300,excellent,6.0,gas,57954.0,automatic,sedan,black,1.0,2018-06-20,15
6,12990,2015.0,toyota camry,excellent,4.0,gas,79212.0,automatic,sedan,white,NaN,2018-12-27,73
7,15990,2013.0,honda pilot,excellent,6.0,gas,109473.0,automatic,SUV,black,1.0,2019-01-07,68
8,11500,2012.0,kia sorento,excellent,4.0,gas,104174.0,automatic,SUV,NaN,1.0,2018-07-16,19
9,9200,2008.0,honda pilot,excellent,NaN,gas,147191.0,automatic,SUV,blue,1.0,2019-02-15,17


**Conclusion:**
The dataset was loaded without any issues. A quick look shows we have both numbers and text — a good mix of features to analyze.

### 1.2 Dataset Overview

To better understand the dataset, I examine:
- The total number of entries and features
- The data types of each column
- The number of missing values

This will help guide the next steps in data cleaning and exploration.

In [18]:
# General info
print(df.info())

# Column names
df.columns.tolist()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB
None


['price',
 'model_year',
 'model',
 'condition',
 'cylinders',
 'fuel',
 'odometer',
 'transmission',
 'type',
 'paint_color',
 'is_4wd',
 'date_posted',
 'days_listed']

**Conclusion:**
There are over 51,000 rows and 13 columns. Some columns contain text, others have numbers. We can already see that a few columns are missing some data.

### 1.3 Unique Values in Each Column

This helps distinguish between categorical and numerical variables, and reveals potential for grouping or encoding.

In [19]:
df.nunique()

price            3443
model_year         68
model             100
condition           6
cylinders           7
fuel                5
odometer        17762
transmission        3
type               13
paint_color        12
is_4wd              1
date_posted       354
days_listed       227
dtype: int64

**Conclusion:**
Some columns like model and paint_color have a lot of unique values. Others, like transmission or fuel, have just a few. This helps us understand what kind of analysis or cleaning might be needed later.

### 1.4 Missing Values

Check for missing data in the dataset.

In [20]:
df.isna().sum()

price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64

**Conclusion:**
A few columns are missing quite a lot of data — especially is_4wd, paint_color, odometer, cylinders, and model_year. We'll need to decide how to handle these gaps later.

### 1.5 Duplicate Rows

Make sure there are no exact duplicates.

In [21]:
df.duplicated().sum()

0

**Conclusion:**
There are no exact duplicate rows in the dataset.

### 1.6 Summary Statistics

Basic statistical overview of numerical features.


In [22]:
df.describe()

,price,model_year,cylinders,odometer,is_4wd,days_listed
count,51525.000000,47906.000000,46265.000000,43633.000000,25572.0,51525.00000
mean,12132.464920,2009.750470,6.125235,115553.461738,1.0,39.55476
std,10040.803015,6.282065,1.660360,65094.611341,0.0,28.20427
min,1.000000,1908.000000,3.000000,0.000000,1.0,0.00000
25%,5000.000000,2006.000000,4.000000,70000.000000,1.0,19.00000
50%,9000.000000,2011.000000,6.000000,113000.000000,1.0,33.00000
75%,16839.000000,2014.000000,8.000000,155000.000000,1.0,53.00000
max,375000.000000,2019.000000,12.000000,990000.000000,1.0,271.00000


**Conclusion:**
There are some strange values in the data, like very low prices or very old cars (from 1908!). This means we'll probably need to clean up outliers before analysis.

### 1.7 Initial Observations

- Dataset contains 51,525 entries and 13 columns.
- It includes both numerical and categorical variables.
- Some columns contain missing values: `model_year`, `cylinders`, `odometer`, `paint_color`, `is_4wd`.
- `is_4wd` is likely binary, with missing values for non-4WD cars.
- Several features (`model`, `type`, `paint_color`) contain many unique categories — grouping may help.
- No duplicate rows detected.
- Column names are in `snake_case`.
- `date_posted` is of type `object` and will be converted to `datetime` later.

## 2. Data Cleaning

### 2.1 Handling Missing Values

In [23]:
# Check missing values again
df.isna().sum()

price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64

**Strategy:**

`model_year`, `cylinders`, `odometer`, `paint_color`, `is_4wd`: we'll drop rows where these are missing, since they are important for our analysis and hard to guess.

We'll keep only listings that have all the essential numeric fields filled.

**Why I removed rows with missing values**
Some columns like `price`, `model_year`, `odometer`, `cylinders`, and `is_4wd` are crucial for my analysis — they directly relate to the research questions I'm trying to answer.
If values in these fields are missing, it's hard to draw reliable conclusions. Since the number of such rows was relatively small, I decided to remove them to keep the dataset clean and consistent.

In [24]:
# Drop rows with critical missing values
df = df.dropna(subset=['model_year', 'cylinders', 'odometer', 'paint_color', 'is_4wd'])

# Check shape after dropping
df.shape

(14852, 13)

 ### 2.2 Converting Data Types

In [25]:
# Convert 'date_posted' to datetime
df['date_posted'] = pd.to_datetime(df['date_posted'])

# Convert 'model_year' to integer
df['model_year'] = df['model_year'].astype(int)

# Convert 'cylinders' to integer
df['cylinders'] = df['cylinders'].astype(int)

# Convert 'is_4wd' to integer (0 or 1)
df['is_4wd'] = df['is_4wd'].astype(int)

### 2.3 Fixing Outliers

We saw unusual values like:

- price as low as 105
- odometer with some very low and very high values
- model_year as early as 1908

We’ll filter out unrealistic entries based on domain knowledge.

In [26]:
# Remove listings with prices below $500 or above $100,000
df = df[(df['price'] >= 500) & (df['price'] <= 100000)]

# Keep odometer between 1,000 and 300,000 miles
df = df[(df['odometer'] >= 1000) & (df['odometer'] <= 300000)]

# Keep model_year between 1980 and 2022
df = df[(df['model_year'] >= 1980) & (df['model_year'] <= 2022)]

**Why I removed outliers**

I also excluded extreme values in numeric columns like `price` and `odometer`. These values likely represent errors (like test ads or typos) or very rare cases that don’t reflect typical market behavior.
Keeping them could distort the analysis, affect the visualizations, and lead to misleading insights. Removing them helped me focus on the core data distribution.

### 2.4 Final Check

In [27]:
# Make sure everything looks good
df.info()
df.describe()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 14107 entries, 5 to 51515
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         14107 non-null  int64         
 1   model_year    14107 non-null  int64         
 2   model         14107 non-null  object        
 3   condition     14107 non-null  object        
 4   cylinders     14107 non-null  int64         
 5   fuel          14107 non-null  object        
 6   odometer      14107 non-null  float64       
 7   transmission  14107 non-null  object        
 8   type          14107 non-null  object        
 9   paint_color   14107 non-null  object        
 10  is_4wd        14107 non-null  int64         
 11  date_posted   14107 non-null  datetime64[ns]
 12  days_listed   14107 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(5), object(6)
memory usage: 1.5+ MB


price           0
model_year      0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
transmission    0
type            0
paint_color     0
is_4wd          0
date_posted     0
days_listed     0
dtype: int64

**Conclusion:**

- All missing values have been handled.
- Data types are properly formatted.
- Dataset is clean and ready for exploration.

## 3. Exploratory Data Analysis and Hypothesis Testing

In this section, we analyze the cleaned dataset to identify patterns and relationships between key features and the target variables — `price` and `days_listed`. The analysis is structured around the hypotheses defined earlier in the project.

### 3.1. Hypothesis 1: Vehicles with higher mileage (odometer) have lower prices

In [28]:
import plotly.express as px

# Create a density heatmap (as an alternative to hexbin)
fig = px.density_heatmap(
    df,
    x='odometer',
    y='price',
    nbinsx=50,
    nbinsy=50,
    color_continuous_scale='Blues',
    title='Price vs. Odometer (Density Heatmap)'
)

# Update axis labels
fig.update_layout(
    xaxis_title='Odometer (Mileage)',
    yaxis_title='Price ($)',
    coloraxis_colorbar=dict(title='Number of Listings')
)

fig.show()

As shown in the heatmap, there is a clear negative relationship between mileage and price. The red regression line confirms this trend: cars with higher odometer readings tend to have lower prices.

This result makes sense intuitively — the more a car has been driven, the more wear and tear it likely has, which lowers its market value.

The plot also shows that the majority of listings are clustered in the range of 50,000 to 150,000 miles and prices between $5,000 and $20,000. This suggests that most used vehicles on the market fall within this range.

**Conclusion:** The data supports the hypothesis that vehicles with higher mileage tend to have lower prices.

### 3.2. Hypothesis 2: Newer vehicles (model_year) are generally more expensive

In [31]:
import plotly.express as px

fig = px.box(
    df,
    x='model_year',
    y='price',
    title='Price Distribution by Model Year',
    labels={'model_year': 'Model Year', 'price': 'Price ($)'}
)

fig.update_layout(
    xaxis_title='Model Year',
    yaxis_title='Price ($)',
    showlegend=False
)

fig.show()

This boxplot shows the distribution of vehicle prices for each model year: 

- The bold horizontal line inside each box represents the median price — the middle value for that year.
- The colored box shows the interquartile range (IQR) — from the 25th percentile (bottom of the box) to the 75th percentile (top of the box). This means 50% of all listings fall inside this range.
- The "whiskers" (lines extending from the box) show the general spread of the data, up to 1.5 times the IQR. These help visualize the typical price range without the outliers.
- The small circles beyond the whiskers are outliers — listings with prices that are unusually high or low compared to most others in that year.

**Overall, we can clearly see a positive correlation between model year and price: newer vehicles tend to be more expensive. In addition, the variation in prices (both typical and extreme) increases in more recent years, possibly due to broader model ranges and advanced features.**

### 3.3. Hypothesis 3: Cars in better condition are both priced higher and sell more quickly

In [ ]:
import plotly.express as px

# Overlaid histogram of price by condition
fig = px.histogram(
    df,
    x='price',
    color='condition',
    barmode='overlay',
    nbins=50,
    opacity=0.6,
    title='Price Distribution by Vehicle Condition',
    labels={'price': 'Price ($)', 'condition': 'Condition'}
)

# Update layout
fig.update_layout(
    xaxis_title='Price ($)',
    yaxis_title='Number of Listings',
    legend_title='Condition'
)

fig.show()

**This histogram shows how vehicle price varies depending on its condition:**

- Cars in excellent and like new condition tend to have higher prices, with distributions skewed toward the upper price range.
- New cars are rare but also appear in the highest price range.
- Vehicles in good, fair, and especially salvage condition show significantly lower price distributions.
- The peak of each distribution shifts left (toward cheaper prices) as the condition worsens.
- The spread is wider for higher-quality cars, likely reflecting a broader range of models and pricing flexibility in those categories.

In [ ]:
import plotly.express as px

fig = px.histogram(
    df, 
    x='days_listed', 
    color='condition', 
    barmode='overlay',
    nbins=50,
    title='Listing Duration by Vehicle Condition',
    labels={'days_listed': 'Days Listed', 'condition': 'Condition'}
)

fig.update_layout(
    xaxis_title='Days Listed',
    yaxis_title='Number of Listings'
)

fig.show()

**This histogram shows how listing duration (days listed) varies depending on vehicle condition:**

- Most vehicles in excellent and good condition are sold within the first 30–40 days, though some listings stretch well beyond 100 days.
- New and like new cars tend to sell faster — their bars are highest in the early bins and drop sharply afterward.
- Salvage vehicles often sell quickly too — possibly due to low prices — but the volume is small.
- Listings for fair condition vehicles show a more gradual decline, indicating a broader range of time on market.

As with price, better condition generally correlates with quicker turnover — but not perfectly.

**Conclusion**

The data supports the first part of the hypothesis: vehicles in better condition tend to have higher prices. Price distributions are clearly skewed upward for excellent, like new, and new vehicles.

The second part — that condition influences how quickly cars are sold — is only partially supported. While cars in better condition (especially new or like new) often sell faster, the relationship is not strictly linear. Some salvage vehicles sell quickly, and excellent/good cars may remain listed for extended periods, likely depending on price competitiveness or specific buyer preferences.

Overall conclusion:
Vehicle condition is a strong predictor of price, and a moderate predictor of listing duration, though the latter is influenced by other factors such as pricing, model, or demand fluctuations.

### 3.4. Hypothesis 4: Vehicle type influences pricing — e.g., SUVs and trucks may be priced higher than sedans.

In [ ]:
import plotly.express as px
import pandas as pd

# Group and sort average prices by vehicle type
avg_price_by_type = df.groupby('type')['price'].mean().reset_index()
avg_price_by_type = avg_price_by_type.sort_values(by='price', ascending=False)

# Round the average prices to whole numbers
avg_price_by_type['price'] = avg_price_by_type['price'].round(0)

# Create the bar chart
fig = px.bar(
    avg_price_by_type,
    x='type',
    y='price',
    text='price',  # show price on each bar
    title='Average Price by Vehicle Type',
    labels={'price': 'Average Price ($)', 'type': 'Vehicle Type'}
)

# Update the chart layout and appearance
fig.update_traces(
    texttemplate='%{text:.0f}',  # format text as integer
    textposition='outside',      # place labels outside bars
    marker_color='#4C6EF5'   
)

fig.update_layout(
    xaxis_title='Vehicle Type',
    yaxis_title='Average Price ($)',
    yaxis_tickformat=',.0f',     # format y-axis ticks with commas
    xaxis_tickangle=-45,         # rotate x-axis labels for readability
    uniformtext_minsize=8,
    uniformtext_mode='hide'
)

fig.show()

**Conclusion:**

The analysis confirms that vehicle type has a significant impact on price. Trucks and pickups have the highest average prices, followed by coupes and offroad vehicles. In contrast, hatchbacks, mini-vans, and sedans tend to be the most affordable.

This pattern likely reflects differences in size, utility, and market demand for specific vehicle types. For example, trucks and pickups are often newer, more powerful, and equipped with additional features, while hatchbacks and sedans are generally smaller and more economical.

### 3.5. Hypothesis 5: Fuel type affects price and sale duration.

In [ ]:
import plotly.express as px

# --- Average price by fuel type ---
avg_price_by_fuel = df.groupby('fuel')['price'].mean().reset_index()

fig_price = px.bar(
    avg_price_by_fuel.sort_values(by='price', ascending=False),
    x='fuel',
    y='price',
    title='Average Price by Fuel Type',
    labels={'fuel': 'Fuel Type', 'price': 'Average Price ($)'},
    text_auto='.0f',
    color_discrete_sequence=['#4C6EF5']
)

fig_price.update_layout(
    width=800,
    height=400,
    yaxis_title='Average Price ($)'
)

fig_price.show()

# --- Average days listed by fuel type ---
avg_days_by_fuel = df.groupby('fuel')['days_listed'].mean().reset_index()

fig_days = px.bar(
    avg_days_by_fuel.sort_values(by='days_listed', ascending=False),
    x='fuel',
    y='days_listed',
    title='Average Days Listed by Fuel Type',
    labels={'fuel': 'Fuel Type', 'days_listed': 'Average Days Listed'},
    text_auto='.0f',
    color_discrete_sequence=['#4C6EF5']
)

fig_days.update_layout(
    width=800,
    height=400,
    yaxis_title='Average Days Listed'
)

fig_days.show()

**Findings:**

1. Average Price by Fuel Type:

- Diesel vehicles have the highest average price (~$25,124), followed by "other" and gas.
- Hybrid cars are the cheapest, with an average price of ~$8,757.

2. Average Days Listed by Fuel Type:

- Contrary to what one might expect, hybrid vehicles, despite being cheapest, take the longest to sell (49 days on average).
- Other, gas, and diesel vehicles are listed for a similar and shorter average time (38–40 days).

**Conclusion:**

The hypothesis is partially supported:

- Fuel type clearly affects car prices — diesel cars are significantly more expensive, while hybrids are the cheapest.
- However, selling time (days listed) does not follow the same trend. Hybrid cars, despite being cheaper, take the longest to sell, while more expensive diesel and gas cars sell faster on average.

This suggests that price alone does not determine selling speed, and other factors — such as demand for specific fuel types — may influence time on the market.



## 4. Summary of Key Findings

- Vehicle condition, mileage, and model year significantly impact pricing.
- SUVs and trucks tend to have higher average prices than sedans.
- Electric and hybrid cars are generally priced higher, though demand varies.
- While newer cars tend to sell faster, listing duration also depends on vehicle type and condition.
- There's no simple linear relationship between a single feature and sale speed—multiple factors interact.

These insights can help improve pricing strategy and inventory management for used car marketplaces.
